In [1]:
import numpy as np

'''
데이터 생성
'''
rng = np.random.RandomState(123)

d = 2
N = 10

mean = 5

x1 = rng.randn(N, d) + np.array([0, 0])
x2 = rng.randn(N, d) + np.array([mean, mean])

x = np.concatenate((x1, x2), axis=0)

'''
단순 퍼셉트론
'''
w = np.zeros(d)
b = 0


def y(x):
    return step(np.dot(w, x) + b)


def step(x):
    return 1 * (x > 0)


def t(i):
    if i < N:
        return 0
    else:
        return 1


while True:
    classified = True
    for i in range(N * 2):
        delta_w = (t(i) - y(x[i])) * x[i]
        delta_b = (t(i) - y(x[i]))
        w += delta_w
        b += delta_b
        classified *= all(delta_w == 0) * (delta_b == 0)
    if classified:
        break

print('w:', w)
print('b:', b)

print('\nTest:')
print(y([0, 0]))  # => 0
print(y([5, 5]))  # => 1


w: [2.14037745 1.2763927 ]
b: -9

Test:
0
1


In [4]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


'''
모델 설정
'''
tf.set_random_seed(0)  # 난수 시드

w = tf.Variable(tf.zeros([2, 1]))
b = tf.Variable(tf.zeros([1]))

x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])
y = tf.nn.sigmoid(tf.matmul(x, w) + b)

cross_entropy = - tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

'''
모델 학습
'''
# OR 게이트
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

# 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 학습
for epoch in range(200):
    sess.run(train_step, feed_dict={
        x: X,
        t: Y
    })

'''
학습 결과를 확인한다
'''
classified = correct_prediction.eval(session=sess, feed_dict={
    x: X,
    t: Y
})
prob = y.eval(session=sess, feed_dict={
    x: X
})

print('classified:')
print(classified)
print()
print('output probability:')
print(prob)


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use `tf.cast` instead.
classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability:
[[0.22355038]
 [0.9142595 ]
 [0.9142595 ]
 [0.9974742 ]]


In [5]:
import numpy as np
#import tensorflow as tf
from sklearn.utils import shuffle

np.random.seed(0)
tf.set_random_seed(0)

M = 2      # 입력 데이터의 차원
K = 3      # 클래스 수
n = 100    # 각 클래스에 있는 데이터 수
N = n * K  # 전체 데이터 수

'''
데이터 생성
'''
X1 = np.random.randn(n, M) + np.array([0, 10])
X2 = np.random.randn(n, M) + np.array([5, 5])
X3 = np.random.randn(n, M) + np.array([10, 0])
Y1 = np.array([[1, 0, 0] for i in range(n)])
Y2 = np.array([[0, 1, 0] for i in range(n)])
Y3 = np.array([[0, 0, 1] for i in range(n)])

X = np.concatenate((X1, X2, X3), axis=0)
Y = np.concatenate((Y1, Y2, Y3), axis=0)

'''
모델 생성
'''
W = tf.Variable(tf.zeros([M, K]))
b = tf.Variable(tf.zeros([K]))

x = tf.placeholder(tf.float32, shape=[None, M])
t = tf.placeholder(tf.float32, shape=[None, K])
y = tf.nn.softmax(tf.matmul(x, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y),
                               reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))

'''
모델을 학습시킨다
'''
# 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 50  # 미니배치의 크기
n_batches = N // batch_size

# 미니배치 학습
for epoch in range(20):
    X_, Y_ = shuffle(X, Y)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end]
        })

'''
학습 결과를 확인한다
'''
X_, Y_ = shuffle(X, Y)

classified = correct_prediction.eval(session=sess, feed_dict={
    x: X_[0:10],
    t: Y_[0:10]
})
prob = y.eval(session=sess, feed_dict={
    x: X_[0:10]
})

print('classified:')
print(classified)
print()
print('output probability:')
print(prob)


classified:
[ True  True  True  True  True  True  True  True  True  True]

output probability:
[[9.9868268e-01 1.3173160e-03 2.5878538e-10]
 [3.4313095e-03 9.6904927e-01 2.7519440e-02]
 [9.6915740e-01 3.0842537e-02 5.0909865e-08]
 [1.9351479e-02 9.7068453e-01 9.9640097e-03]
 [4.1215809e-09 8.5910317e-03 9.9140900e-01]
 [1.7154520e-02 9.7633582e-01 6.5097027e-03]
 [2.3086071e-07 4.2457711e-02 9.5754212e-01]
 [7.2534569e-08 8.8783968e-03 9.9112153e-01]
 [8.4302730e-12 1.4183836e-04 9.9985814e-01]
 [9.9589419e-01 4.1058036e-03 2.6871794e-09]]


In [1]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


tf.set_random_seed(0)

'''
데이터를 생성한다
'''
# XOR 게이트
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

'''
모델을 생성한다
'''
x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])

# 입력층-은닉층
W = tf.Variable(tf.truncated_normal([2, 2]))
b = tf.Variable(tf.zeros([2]))
h = tf.nn.sigmoid(tf.matmul(x, W) + b)

# 은닉층-출력층
V = tf.Variable(tf.truncated_normal([2, 1]))
c = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(h, V) + c)

cross_entropy = - tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

'''
모델을 학습시킨다
'''
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(4000):
    sess.run(train_step, feed_dict={
        x: X,
        t: Y
    })
    if epoch % 1000 == 0:
        print('epoch:', epoch)

'''
학습 결과를 확인한다
'''
classified = correct_prediction.eval(session=sess, feed_dict={
    x: X,
    t: Y
})
prob = y.eval(session=sess, feed_dict={
    x: X
})

print('classified:')
print(classified)
print()
print('output probability:')
print(prob)


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use `tf.cast` instead.
epoch: 0
epoch: 1000
epoch: 2000
epoch: 3000
classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability:
[[0.00668001]
 [0.9926348 ]
 [0.99263966]
 [0.01130182]]


In [2]:
import numpy as np
#import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)
tf.set_random_seed(1234)

'''
데이터를 생성한다
'''
N = 300  # 전체 데이터 개수
X, y = datasets.make_moons(N, noise=0.3)
Y = y.reshape(N, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)


'''
모델을 생성한다
'''
num_hidden = 3  # 은닉층의 차원수
# num_hidden = 2

x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])

# 입력층-은닉층
W = tf.Variable(tf.truncated_normal([2, num_hidden]))
b = tf.Variable(tf.zeros([num_hidden]))
h = tf.nn.sigmoid(tf.matmul(x, W) + b)

# 은닉층-출력층
V = tf.Variable(tf.truncated_normal([num_hidden, 1]))
c = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(h, V) + c)

cross_entropy = - tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

'''
모델을 학습시킨다
'''
batch_size = 20
n_batches = N // batch_size

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(500):
    X_, Y_ = shuffle(X_train, Y_train)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end]
        })

'''
예측 정확도를 평가한다
'''
accuracy_rate = accuracy.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test
})
print('accuracy: ', accuracy_rate)


accuracy:  0.95


In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

np.random.seed(0)  # 난수 시드

'''
모델 설정
'''
model = Sequential([
    # Dense(input_dim=2, output_dim=1),  # Keras 1
    Dense(input_dim=2, units=1),         # Keras 2
    Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1))

'''
모델 학습
'''
# OR 게이트
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

# model.fit(X, Y, nb_epoch=200, batch_size=1)  # Keras 1
model.fit(X, Y, epochs=200, batch_size=1)      # Keras 2

'''
학습 결과를 확인한다
'''
classes = model.predict_classes(X, batch_size=1)
prob = model.predict_proba(X, batch_size=1)

print('classified:')
print(Y == classes)
print()
print('output probability:')
print(prob)


Train on 4 samples
Epoch 1/200
4/4 [==============================] - 0s 697us/sample - loss: 1.2715
Epoch 2/200
4/4 [==============================] - 0s 500us/sample - loss: 1.0939
Epoch 3/200
4/4 [==============================] - 0s 750us/sample - loss: 0.9545
Epoch 4/200
4/4 [==============================] - 0s 500us/sample - loss: 0.8476
Epoch 5/200
4/4 [==============================] - 0s 750us/sample - loss: 0.7680
Epoch 6/200
4/4 [==============================] - 0s 999us/sample - loss: 0.7084
Epoch 7/200
4/4 [==============================] - 0s 750us/sample - loss: 0.6617
Epoch 8/200
4/4 [==============================] - 0s 500us/sample - loss: 0.6259
Epoch 9/200
4/4 [==============================] - 0s 750us/sample - loss: 0.5981
Epoch 10/200
4/4 [==============================] - 0s 750us/sample - loss: 0.5759
Epoch 11/200
4/4 [==============================] - 0s 749us/sample - loss: 0.5577
Epoch 12/200
4/4 [==============================] - 0s 999us/sample - loss: 0

4/4 [==============================] - 0s 999us/sample - loss: 0.2099
Epoch 100/200
4/4 [==============================] - 0s 749us/sample - loss: 0.2084
Epoch 101/200
4/4 [==============================] - 0s 750us/sample - loss: 0.2069
Epoch 102/200
4/4 [==============================] - 0s 750us/sample - loss: 0.2054
Epoch 103/200
4/4 [==============================] - 0s 749us/sample - loss: 0.2039
Epoch 104/200
4/4 [==============================] - 0s 750us/sample - loss: 0.2024
Epoch 105/200
4/4 [==============================] - 0s 999us/sample - loss: 0.2010
Epoch 106/200
4/4 [==============================] - 0s 749us/sample - loss: 0.1996
Epoch 107/200
4/4 [==============================] - 0s 750us/sample - loss: 0.1982
Epoch 108/200
4/4 [==============================] - 0s 999us/sample - loss: 0.1968
Epoch 109/200
4/4 [==============================] - 0s 749us/sample - loss: 0.1954
Epoch 110/200
4/4 [==============================] - 0s 750us/sample - loss: 0.1941
Epoch 

4/4 [==============================] - 0s 749us/sample - loss: 0.1201
Epoch 197/200
4/4 [==============================] - 0s 500us/sample - loss: 0.1196
Epoch 198/200
4/4 [==============================] - 0s 750us/sample - loss: 0.1190
Epoch 199/200
4/4 [==============================] - 0s 750us/sample - loss: 0.1185
Epoch 200/200
4/4 [==============================] - 0s 752us/sample - loss: 0.1180
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
Please use `model.predict()` instead.
classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability:
[[0.2383

In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from sklearn.utils import shuffle

np.random.seed(0)  # 난수 시드

M = 2      # 입력 데이터의 차원
K = 3      # 클래스 수
n = 100    # 각 클래스에 있는 데이터 수
N = n * K  # 모든 데이터 수

'''
데이터 생성
'''
X1 = np.random.randn(n, M) + np.array([0, 10])
X2 = np.random.randn(n, M) + np.array([5, 5])
X3 = np.random.randn(n, M) + np.array([10, 0])
Y1 = np.array([[1, 0, 0] for i in range(n)])
Y2 = np.array([[0, 1, 0] for i in range(n)])
Y3 = np.array([[0, 0, 1] for i in range(n)])

X = np.concatenate((X1, X2, X3), axis=0)
Y = np.concatenate((Y1, Y2, Y3), axis=0)

'''
모델을 설정한다
'''
model = Sequential()
model.add(Dense(input_dim=M, units=K))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1))

'''
모델을 학습시킨다
'''
minibatch_size = 50
model.fit(X, Y, epochs=20, batch_size=minibatch_size)

'''
학습 결과를 확인한다
'''
X_, Y_ = shuffle(X, Y)
classes = model.predict_classes(X_[0:10], batch_size=minibatch_size)
prob = model.predict_proba(X_[0:10], batch_size=minibatch_size)
print('classified:')
print(np.argmax(model.predict(X_[0:10]), axis=1) == classes)
print()
print('output probability:')
print(prob)


Train on 300 samples
Epoch 1/20
300/300 [==============================] - 0s 13us/sample - loss: 1.8088
Epoch 2/20
300/300 [==============================] - 0s 10us/sample - loss: 0.2513
Epoch 3/20
300/300 [==============================] - 0s 13us/sample - loss: 0.1671
Epoch 4/20
300/300 [==============================] - 0s 10us/sample - loss: 0.1277
Epoch 5/20
300/300 [==============================] - 0s 17us/sample - loss: 0.1045
Epoch 6/20
300/300 [==============================] - 0s 13us/sample - loss: 0.0901
Epoch 7/20
300/300 [==============================] - 0s 17us/sample - loss: 0.0776
Epoch 8/20
300/300 [==============================] - 0s 10us/sample - loss: 0.0712
Epoch 9/20
300/300 [==============================] - 0s 13us/sample - loss: 0.0639
Epoch 10/20
300/300 [==============================] - 0s 10us/sample - loss: 0.0583
Epoch 11/20
300/300 [==============================] - 0s 13us/sample - loss: 0.0540
Epoch 12/20
300/300 [==============================] 

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

np.random.seed(123)

'''
데이터를 생성한다
'''
# XORゲート
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

'''
모델을 설정한다
'''
model = Sequential()

# 입력층-은닉층
model.add(Dense(input_dim=2, units=2))
model.add(Activation('sigmoid'))

# 은닉층-출력층
model.add(Dense(units=1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1))

'''
모델을 학습시킨다
'''
model.fit(X, Y, epochs=4000, batch_size=4)

'''
학습 결과를 확인한다
'''
classes = model.predict_classes(X, batch_size=4)
prob = model.predict_proba(X, batch_size=4)

print('classified:')
print(Y == classes)
print()
print('output probability:')
print(prob)


Train on 4 samples
Epoch 1/4000
4/4 [==============================] - 0s 260us/sample - loss: 0.6885
Epoch 2/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6884
Epoch 3/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6883
Epoch 4/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6882
Epoch 5/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6881
Epoch 6/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6880
Epoch 7/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6879
Epoch 8/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.6879
Epoch 9/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6878
Epoch 10/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6878
Epoch 11/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6877
Epoch 12/4000
4/4 [==============================] - 0s 250us/samp

4/4 [==============================] - 0s 500us/sample - loss: 0.6854
Epoch 99/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6854
Epoch 100/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6853
Epoch 101/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6853
Epoch 102/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6853
Epoch 103/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6853
Epoch 104/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6852
Epoch 105/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6852
Epoch 106/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6852
Epoch 107/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6852
Epoch 108/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.6851
Epoch 109/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6

Epoch 195/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6827
Epoch 196/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6826
Epoch 197/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6826
Epoch 198/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6826
Epoch 199/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6826
Epoch 200/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6825
Epoch 201/4000
4/4 [==============================] - 0s 499us/sample - loss: 0.6825
Epoch 202/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6825
Epoch 203/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6824
Epoch 204/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6824
Epoch 205/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6824
Epoch 206/4000
4/4 [==============================] - 0s 250us/sa

4/4 [==============================] - 0s 250us/sample - loss: 0.6794
Epoch 292/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.6793
Epoch 293/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6793
Epoch 294/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6793
Epoch 295/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6792
Epoch 296/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6792
Epoch 297/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6791
Epoch 298/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6791
Epoch 299/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6791
Epoch 300/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6790
Epoch 301/4000
4/4 [==============================] - 0s 0s/sample - loss: 0.6790
Epoch 302/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.678

4/4 [==============================] - 0s 250us/sample - loss: 0.6753
Epoch 389/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6752
Epoch 390/4000
4/4 [==============================] - 0s 0s/sample - loss: 0.6752
Epoch 391/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6751
Epoch 392/4000
4/4 [==============================] - 0s 0s/sample - loss: 0.6751
Epoch 393/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6750
Epoch 394/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6750
Epoch 395/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6749
Epoch 396/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6749
Epoch 397/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.6749
Epoch 398/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6748
Epoch 399/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.6748
E

...

4/4 [==============================] - 0s 250us/sample - loss: 0.1215
Epoch 3537/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1214
Epoch 3538/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1213
Epoch 3539/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1212
Epoch 3540/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1211
Epoch 3541/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1210
Epoch 3542/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.1208
Epoch 3543/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1207
Epoch 3544/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.1206
Epoch 3545/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1205
Epoch 3546/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1204
Epoch 3547/4000
4/4 [==============================] - 0s 250us/sample

4/4 [==============================] - 0s 500us/sample - loss: 0.1116
Epoch 3633/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1115
Epoch 3634/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.1114
Epoch 3635/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1113
Epoch 3636/4000
4/4 [==============================] - 0s 249us/sample - loss: 0.1112
Epoch 3637/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1111
Epoch 3638/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1110
Epoch 3639/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1109
Epoch 3640/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1108
Epoch 3641/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.1107
Epoch 3642/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1106
Epoch 3643/4000
4/4 [==============================] - 0s 250us/sample

4/4 [==============================] - 0s 250us/sample - loss: 0.1029
Epoch 3729/4000
4/4 [==============================] - 0s 0s/sample - loss: 0.1028
Epoch 3730/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1027
Epoch 3731/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1026
Epoch 3732/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1026
Epoch 3733/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1025
Epoch 3734/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1024
Epoch 3735/4000
4/4 [==============================] - 0s 0s/sample - loss: 0.1023
Epoch 3736/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1022
Epoch 3737/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1021
Epoch 3738/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.1021
Epoch 3739/4000
4/4 [==============================] - 0s 250us/sample - los

Epoch 3824/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0953
Epoch 3825/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0952
Epoch 3826/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0952
Epoch 3827/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0951
Epoch 3828/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.0950
Epoch 3829/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0949
Epoch 3830/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0949
Epoch 3831/4000
4/4 [==============================] - 0s 0s/sample - loss: 0.0948
Epoch 3832/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0947
Epoch 3833/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0946
Epoch 3834/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.0946
Epoch 3835/4000
4/4 [==============================] - 0s

4/4 [==============================] - 0s 500us/sample - loss: 0.0887
Epoch 3920/4000
4/4 [==============================] - 0s 499us/sample - loss: 0.0886
Epoch 3921/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0886
Epoch 3922/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.0885
Epoch 3923/4000
4/4 [==============================] - ETA: 0s - loss: 0.088 - 0s 250us/sample - loss: 0.0885
Epoch 3924/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0884
Epoch 3925/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0883
Epoch 3926/4000
4/4 [==============================] - 0s 500us/sample - loss: 0.0883
Epoch 3927/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0882
Epoch 3928/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0881
Epoch 3929/4000
4/4 [==============================] - 0s 250us/sample - loss: 0.0881
Epoch 3930/4000
4/4 [=========================

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)

'''
데이터를 생성한다
'''
N = 300
X, y = datasets.make_moons(N, noise=0.3)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

'''
모델을 생성한다
'''
model = Sequential()
model.add(Dense(3, input_dim=2))
model.add(Activation('sigmoid'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=0.05),
              metrics=['accuracy'])

'''
모델을 학습시킨다
'''
model.fit(X_train, y_train, epochs=500, batch_size=20)

'''
예측 정확도를 평가한다
'''
loss_and_metrics = model.evaluate(X_test, y_test)
print(loss_and_metrics)


Train on 240 samples
Epoch 1/500
240/240 [==============================] - 0s 29us/sample - loss: 0.6382 - acc: 0.7875
Epoch 2/500
240/240 [==============================] - 0s 37us/sample - loss: 0.6331 - acc: 0.7833
Epoch 3/500
240/240 [==============================] - 0s 37us/sample - loss: 0.6282 - acc: 0.7958
Epoch 4/500
240/240 [==============================] - 0s 33us/sample - loss: 0.6233 - acc: 0.8083
Epoch 5/500
240/240 [==============================] - 0s 33us/sample - loss: 0.6187 - acc: 0.8042
Epoch 6/500
240/240 [==============================] - 0s 42us/sample - loss: 0.6134 - acc: 0.8125
Epoch 7/500
240/240 [==============================] - 0s 33us/sample - loss: 0.6087 - acc: 0.8083
Epoch 8/500
240/240 [==============================] - 0s 29us/sample - loss: 0.6036 - acc: 0.8083
Epoch 9/500
240/240 [==============================] - 0s 33us/sample - loss: 0.5989 - acc: 0.8083
Epoch 10/500
240/240 [==============================] - 0s 33us/sample - loss: 0.5941 - 

240/240 [==============================] - 0s 33us/sample - loss: 0.4059 - acc: 0.8125
Epoch 83/500
240/240 [==============================] - 0s 29us/sample - loss: 0.4053 - acc: 0.8125
Epoch 84/500
240/240 [==============================] - 0s 33us/sample - loss: 0.4050 - acc: 0.8167
Epoch 85/500
240/240 [==============================] - 0s 33us/sample - loss: 0.4037 - acc: 0.8167
Epoch 86/500
240/240 [==============================] - 0s 25us/sample - loss: 0.4032 - acc: 0.8167
Epoch 87/500
240/240 [==============================] - 0s 33us/sample - loss: 0.4028 - acc: 0.8167
Epoch 88/500
240/240 [==============================] - 0s 29us/sample - loss: 0.4018 - acc: 0.8167
Epoch 89/500
240/240 [==============================] - 0s 25us/sample - loss: 0.4015 - acc: 0.8083
Epoch 90/500
240/240 [==============================] - 0s 33us/sample - loss: 0.4009 - acc: 0.8125
Epoch 91/500
240/240 [==============================] - 0s 25us/sample - loss: 0.4002 - acc: 0.8125
Epoch 92/500


240/240 [==============================] - 0s 33us/sample - loss: 0.3822 - acc: 0.8125
Epoch 164/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3821 - acc: 0.8083
Epoch 165/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3818 - acc: 0.8083
Epoch 166/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3819 - acc: 0.8083
Epoch 167/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3818 - acc: 0.8083
Epoch 168/500
240/240 [==============================] - 0s 25us/sample - loss: 0.3815 - acc: 0.8083
Epoch 169/500
240/240 [==============================] - 0s 37us/sample - loss: 0.3815 - acc: 0.8083
Epoch 170/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3814 - acc: 0.8125
Epoch 171/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3815 - acc: 0.8125
Epoch 172/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3816 - acc: 0.8125
Epoc

240/240 [==============================] - 0s 33us/sample - loss: 0.3785 - acc: 0.8125
Epoch 244/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3782 - acc: 0.8125
Epoch 245/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3786 - acc: 0.8125
Epoch 246/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3785 - acc: 0.8125
Epoch 247/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3787 - acc: 0.8125
Epoch 248/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3782 - acc: 0.8125
Epoch 249/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3784 - acc: 0.8125
Epoch 250/500
240/240 [==============================] - 0s 25us/sample - loss: 0.3786 - acc: 0.8125
Epoch 251/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3783 - acc: 0.8125
Epoch 252/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3784 - acc: 0.8125
Epoc

240/240 [==============================] - 0s 25us/sample - loss: 0.3771 - acc: 0.8125
Epoch 324/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3770 - acc: 0.8125
Epoch 325/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3770 - acc: 0.8125
Epoch 326/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3771 - acc: 0.8125
Epoch 327/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3774 - acc: 0.8125
Epoch 328/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3769 - acc: 0.8125
Epoch 329/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3772 - acc: 0.8125
Epoch 330/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3767 - acc: 0.8125
Epoch 331/500
240/240 [==============================] - 0s 25us/sample - loss: 0.3769 - acc: 0.8125
Epoch 332/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3769 - acc: 0.8125
Epoc

240/240 [==============================] - 0s 37us/sample - loss: 0.3763 - acc: 0.8125
Epoch 404/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3759 - acc: 0.8125
Epoch 405/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3759 - acc: 0.8125
Epoch 406/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3760 - acc: 0.8125
Epoch 407/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3763 - acc: 0.8125
Epoch 408/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3759 - acc: 0.8125
Epoch 409/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3761 - acc: 0.8125
Epoch 410/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3762 - acc: 0.8125
Epoch 411/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3759 - acc: 0.8125
Epoch 412/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3759 - acc: 0.8125
Epoc

240/240 [==============================] - 0s 37us/sample - loss: 0.3753 - acc: 0.8125
Epoch 484/500
240/240 [==============================] - 0s 37us/sample - loss: 0.3752 - acc: 0.8125
Epoch 485/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3751 - acc: 0.8125
Epoch 486/500
240/240 [==============================] - 0s 37us/sample - loss: 0.3752 - acc: 0.8125
Epoch 487/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3755 - acc: 0.8125
Epoch 488/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3751 - acc: 0.8125
Epoch 489/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3755 - acc: 0.8125
Epoch 490/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3750 - acc: 0.8125
Epoch 491/500
240/240 [==============================] - 0s 33us/sample - loss: 0.3752 - acc: 0.8125
Epoch 492/500
240/240 [==============================] - 0s 29us/sample - loss: 0.3751 - acc: 0.8125
Epoc